In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import networkx as nx
import random
from pyecharts.globals import ThemeType
from pyecharts import options as opts
from pyecharts.charts import Radar,Bar,Line,Page,Grid,Liquid,WordCloud,Polar,Line,Gauge
from pyecharts.faker import Faker
from pyecharts.globals import SymbolType
from random import shuffle

# Read dataset
survey_2019 = pd.read_csv(u'C:\\Users\\蓝尼玛\\Desktop\\StackOverflowSurvey\\developer_survey_2019\\survey_results_public.csv')
schema = pd.read_csv(u'C:\\Users\\蓝尼玛\\Desktop\\StackOverflowSurvey\\developer_survey_2019\\survey_results_schema.csv')

# The transpose data set should have 88883-7548=81335 observations.
#df = pd.get_dummies(survey_2019['DevType'].str.split(';', expand=True).stack()).sum(level=0)

In [2]:
survey_2019.columns

Index(['Respondent', 'MainBranch', 'Hobbyist', 'OpenSourcer', 'OpenSource',
       'Employment', 'Country', 'Student', 'EdLevel', 'UndergradMajor',
       'EduOther', 'OrgSize', 'DevType', 'YearsCode', 'Age1stCode',
       'YearsCodePro', 'CareerSat', 'JobSat', 'MgrIdiot', 'MgrMoney',
       'MgrWant', 'JobSeek', 'LastHireDate', 'LastInt', 'FizzBuzz',
       'JobFactors', 'ResumeUpdate', 'CurrencySymbol', 'CurrencyDesc',
       'CompTotal', 'CompFreq', 'ConvertedComp', 'WorkWeekHrs', 'WorkPlan',
       'WorkChallenge', 'WorkRemote', 'WorkLoc', 'ImpSyn', 'CodeRev',
       'CodeRevHrs', 'UnitTests', 'PurchaseHow', 'PurchaseWhat',
       'LanguageWorkedWith', 'LanguageDesireNextYear', 'DatabaseWorkedWith',
       'DatabaseDesireNextYear', 'PlatformWorkedWith',
       'PlatformDesireNextYear', 'WebFrameWorkedWith',
       'WebFrameDesireNextYear', 'MiscTechWorkedWith',
       'MiscTechDesireNextYear', 'DevEnviron', 'OpSys', 'Containers',
       'BlockchainOrg', 'BlockchainIs', 'BetterLife'

In [20]:
dic = survey_2019['Country'].value_counts().reset_index().to_dict(orient='records')

for x in dic:
    x['name'] = x.pop('index')
    x['value'] = x.pop('Country')


In [21]:
dic

[{'name': 'United States', 'value': 20949},
 {'name': 'India', 'value': 9061},
 {'name': 'Germany', 'value': 5866},
 {'name': 'United Kingdom', 'value': 5737},
 {'name': 'Canada', 'value': 3395},
 {'name': 'France', 'value': 2391},
 {'name': 'Brazil', 'value': 1948},
 {'name': 'Poland', 'value': 1922},
 {'name': 'Australia', 'value': 1903},
 {'name': 'Netherlands', 'value': 1852},
 {'name': 'Russian Federation', 'value': 1694},
 {'name': 'Spain', 'value': 1604},
 {'name': 'Italy', 'value': 1576},
 {'name': 'Sweden', 'value': 1274},
 {'name': 'Switzerland', 'value': 978},
 {'name': 'Israel', 'value': 952},
 {'name': 'Turkey', 'value': 949},
 {'name': 'Pakistan', 'value': 923},
 {'name': 'Ukraine', 'value': 868},
 {'name': 'Austria', 'value': 839},
 {'name': 'Czech Republic', 'value': 764},
 {'name': 'Romania', 'value': 760},
 {'name': 'Iran', 'value': 738},
 {'name': 'Belgium', 'value': 727},
 {'name': 'China', 'value': 664},
 {'name': 'Bulgaria', 'value': 659},
 {'name': 'Mexico', 'val

In [17]:
df = survey_2019.groupby(['Country','Age1stCode']).size().reset_index().dropna()
df.columns = ['Country','Age1stCode','number']
#df['Age1stCode'] = df['Age1stCode'].str.replace('Younger than 5 years',4)

In [18]:
df['Age1stCode'] = df['Age1stCode'].str.replace('Younger than 5 years','4')
df['Age1stCode'] = df['Age1stCode'].str.replace('Older than 85','86')

In [33]:
df['Age1stCode'] = df['Age1stCode'].astype('int')
df['number'] = df['number'].astype('int')
df['Country']  = df['Country'] .astype('str')

In [23]:
#全球平均'Age1stCode'
(df['Age1stCode']*df['number']).sum()/df['number'].sum()

15.409395896569825

In [36]:
# 某一国家

for x in set(df.Country):
    country = x
    counrty_df = df[df['Country'] == country]
    age1cod = (counrty_df['Age1stCode']*counrty_df['number']).sum()/counrty_df['number'].sum()
    print('{"name":"'+x+'",','"value":',age1cod,'},')


{"name":"Taiwan", "value": 16.309782608695652 },
{"name":"Senegal", "value": 22.4 },
{"name":"Trinidad and Tobago", "value": 16.210526315789473 },
{"name":"Uzbekistan", "value": 16.054054054054053 },
{"name":"India", "value": 16.94883405026036 },
{"name":"Togo", "value": 18.75 },
{"name":"Azerbaijan", "value": 17.416666666666668 },
{"name":"United Arab Emirates", "value": 16.36774193548387 },
{"name":"Angola", "value": 18.25 },
{"name":"Haiti", "value": 16.4 },
{"name":"Lithuania", "value": 14.995918367346938 },
{"name":"Panama", "value": 17.272727272727273 },
{"name":"Spain", "value": 15.746221662468514 },
{"name":"The former Yugoslav Republic of Macedonia", "value": 16.96078431372549 },
{"name":"Mauritania", "value": 18.142857142857142 },
{"name":"San Marino", "value": 17.5 },
{"name":"Algeria", "value": 17.623076923076923 },
{"name":"Cyprus", "value": 14.921875 },
{"name":"Singapore", "value": 15.334448160535118 },
{"name":"Burkina Faso", "value": 18.0 },
{"name":"Antigua and Barbud

0       Afghanistan
1       Afghanistan
2       Afghanistan
3       Afghanistan
4       Afghanistan
           ...     
3155       Zimbabwe
3156       Zimbabwe
3157       Zimbabwe
3158       Zimbabwe
3159       Zimbabwe
Name: Country, Length: 3160, dtype: object